In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from xgboost import XGBRegressor
import pickle
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        #X_train, X_test, y_train, y_test = train_test_split(indep_X,dep_Y, test_size = 0.25, random_state = 0)
        #Feature Scaling
        #from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test

def r2_prediction(regressor,X_test,y_test):
     y_pred = regressor.predict(X_test)
     from sklearn.metrics import r2_score
     r2=r2_score(y_test,y_pred)
     return r2
 
def Linear(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2   
    
def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training setC
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     

def random(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 

def xgboost(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from xgboost import XGBRegressor
        regressor = XGBRegressor(n_estimators=100, random_state=42)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 
    
    
def rfeFeature(indep_X,dep_Y,n):
        rfelist=[]
        
        from sklearn.linear_model import LinearRegression
        lin = LinearRegression()
        
        from sklearn.tree import DecisionTreeRegressor
        dec = DecisionTreeRegressor(random_state = 0)
        
        from sklearn.ensemble import RandomForestRegressor
        rf = RandomForestRegressor(n_estimators = 10,random_state = 0)

        from xgboost import XGBRegressor
        xg = XGBRegressor(n_estimators=100, random_state=42)
        
        rfemodellist=[lin,dec,rf,xg] 
        for model in rfemodellist:
             print(f"Performing RFE with model: {model}")
            # Create the RFE model and select the top 'n' features
             log_rfe = RFE(model, n_features_to_select=n)
             # Fit the RFE model
             log_fit = log_rfe.fit(indep_X, dep_Y)
             log_rfe_feature=log_fit.transform(indep_X)
             rfelist.append(log_rfe_feature)
             rfelist
            #Get selected feature columns
             selected_features = indep_X.columns[log_rfe.support_]
          #Print the selected features
             print("Selected features column: ", selected_features)
        return rfelist   
    
def rfe_regression(acclin,accdes,accrf,accxg): 
    
    rfedataframe=pd.DataFrame(index=['Linear','Random','DecisionTree','XgBoost'],columns=['Linear',
                                                                                        'Decision','Random','XgBoost'])
    for number,idex in enumerate(rfedataframe.index):
        rfedataframe['Linear'][idex]=acclin[number]       
        rfedataframe['Decision'][idex]=accdes[number]
        rfedataframe['Random'][idex]=accrf[number]
        rfedataframe['XgBoost'][idex]=accxg[number]
    return rfedataframe



In [2]:
#!pip install xgboost

In [3]:
dataset1=pd.read_csv("preprocessed_house_rent.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True,dtype=int)
indep_X=df2.drop('Total Floors',axis=1)
dep_Y=df2['Total Floors']

In [4]:
df2

,BHK,Rent,Size,City,Bathroom,Area Type_Carpet Area,Area Type_Super Area,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Floor Number,Total Floors
0,2,10000,1100,4,2,0,1,1,0,0,1,0,2
1,2,20000,800,4,1,0,1,1,0,1,0,1,3
2,2,17000,1000,4,1,0,1,1,0,1,0,1,3
3,2,10000,800,4,1,0,1,1,0,0,1,1,2
4,2,7500,850,4,1,1,0,0,0,0,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2,15000,1000,3,2,1,0,1,0,1,0,3,5
4742,3,29000,2000,3,3,0,1,1,0,1,0,1,4
4743,3,35000,1750,3,3,1,0,1,0,1,0,3,5
4744,3,45000,1500,3,2,1,0,0,1,1,0,23,34


In [5]:
rfelist=rfeFeature(indep_X,dep_Y,5)       
acclin=[]
accdes=[]
accrf=[]
accxg=[]


Performing RFE with model: LinearRegression()
Selected features column:  Index(['City', 'Bathroom', 'Area Type_Carpet Area', 'Area Type_Super Area',
       'Floor Number'],
      dtype='object')
Performing RFE with model: DecisionTreeRegressor(random_state=0)
Selected features column:  Index(['Rent', 'Size', 'City', 'Bathroom', 'Floor Number'], dtype='object')
Performing RFE with model: RandomForestRegressor(n_estimators=10, random_state=0)
Selected features column:  Index(['Rent', 'Size', 'City', 'Bathroom', 'Floor Number'], dtype='object')
Performing RFE with model: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning

In [6]:
rfelist

[array([[ 4,  2,  0,  1,  0],
        [ 4,  1,  0,  1,  1],
        [ 4,  1,  0,  1,  1],
        ...,
        [ 3,  3,  1,  0,  3],
        [ 3,  2,  1,  0, 23],
        [ 3,  2,  1,  0,  4]]),
 array([[10000,  1100,     4,     2,     0],
        [20000,   800,     4,     1,     1],
        [17000,  1000,     4,     1,     1],
        ...,
        [35000,  1750,     3,     3,     3],
        [45000,  1500,     3,     2,    23],
        [15000,  1000,     3,     2,     4]]),
 array([[10000,  1100,     4,     2,     0],
        [20000,   800,     4,     1,     1],
        [17000,  1000,     4,     1,     1],
        ...,
        [35000,  1750,     3,     3,     3],
        [45000,  1500,     3,     2,    23],
        [15000,  1000,     3,     2,     4]]),
 array([[ 4,  2,  0,  1,  0],
        [ 4,  1,  0,  0,  1],
        [ 4,  1,  0,  0,  1],
        ...,
        [ 3,  3,  1,  0,  3],
        [ 3,  2,  1,  0, 23],
        [ 3,  2,  1,  1,  4]])]

In [7]:
for i in rfelist:   
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y)  
    r2_lin=Linear(X_train,y_train,X_test)
    acclin.append(r2_lin)
    
    r2_d=Decision(X_train,y_train,X_test)
    accdes.append(r2_d)
    
    r2_r=random(X_train,y_train,X_test)
    accrf.append(r2_r)

    r2_xg=xgboost(X_train,y_train,X_test)
    accxg.append(r2_xg)
    
result=rfe_regression(acclin,accdes,accrf,accxg)



In [8]:
# result
5

5

In [9]:
result

,Linear,Decision,Random,XgBoost
Linear,0.780867,0.714101,0.763694,0.747713
Random,0.76232,0.651376,0.759364,0.753737
DecisionTree,0.76232,0.651376,0.759364,0.753737
XgBoost,0.781171,0.707589,0.763143,0.735776
